# GHZ Circuits. Tensor Network Analysis

This notebook demonstrates the creation, sequential contraction, and parallel contraction of a GHZ circuit using various algorithms such as Girvan–Newman and FlowCutter.

---



## Step 0: Loading software

 First of all, we create a new project to load all the neccesary software

In [1]:
] activate New_Project_on_QXTools;

  Activating project at `C:\Users\Usuario\OneDrive\Escriptori\anaconda_blogs\agost_2023\Novembre_desembre_2024\GitHub_resum_article_Costa_Ballena\New_Project_on_QXTools`


In [ ]:
import Pkg; 
Pkg.add("QXTools")
Pkg.add("QXGraphDecompositions")
Pkg.add("QXZoo")
Pkg.add("DataStructures")
Pkg.add("QXTns")
Pkg.add("NDTensors")
Pkg.add("ITensors")
Pkg.add("LightGraphs")
Pkg.add("PyCall")




   Resolving package versions...
  No Changes to `C:\Users\Usuario\OneDrive\Escriptori\anaconda_blogs\agost_2023\Novembre_desembre_2024\GitHub_resum_article_Costa_Ballena\New_Project_on_QXTools\Project.toml`
  No Changes to `C:\Users\Usuario\OneDrive\Escriptori\anaconda_blogs\agost_2023\Novembre_desembre_2024\GitHub_resum_article_Costa_Ballena\New_Project_on_QXTools\Manifest.toml`


In [2]:
using QXTools
using QXTns
using QXZoo
using PyCall
using QXGraphDecompositions
using LightGraphs
using DataStructures
using TimerOutputs
using ITensors
using LinearAlgebra
using NDTensors


[ Info: OMEinsum loaded the CUDA module successfully


In [3]:
# Load custom functions
include("../src/funcions_article.jl");

## Step 1: Create a GHZ Circuit
We begin by creating a GHZ circuit based on the user-defined number of qubits.

In [4]:
 # --- Step 1: GHZ Creation ---
@info("How many qubits do you want for the GHZ circuit(n)?\n\n")
       
              N = readline() 
              n = parse(Int, N)
       
     
        # Create GHZ circuit
       cct = create_ghz_circuit(n)

       @info(" circuit GHZ with  $(n) qubits created\n\n")

 tnc = convert_to_tnc(cct)  # Convert the GHZ circuit into a tensor network circuit

┌ Info: How many qubits do you want for the GHZ circuit(n)?
└ 


stdin>  999


┌ Info:  circuit GHZ with  999 qubits created
└ 


TensorNetworkCircuit(qubits => 999, gates => 3995)

---

## Step 2: Sequential Contraction Using Girvan-Newman
We perform sequential contraction of the tensor network using the Girvan–Newman algorithm to get a contraction order.

In [5]:
# --- Step 2: Sequential Contraction using Girvan-Newman ---
s1 = Calcul_GN_Sequencial(cct, true)  # Perform sequential contraction
println("Sequential contraction result: ", s1)

────────────────────────────────────────────────────────────────────────────────
                                       Time                    Allocations      
                              ───────────────────────   ────────────────────────
      Tot / % measured:            46.0s /  98.8%           1.70GiB /  98.9%    

Section               ncalls     time    %tot     avg     alloc    %tot      avg
────────────────────────────────────────────────────────────────────────────────
2T. Getting GN plan        1    27.9s   61.4%   27.9s    401MiB   23.3%   401MiB
3T. Final contraction      1    17.0s   37.5%   17.0s   1.24GiB   73.6%  1.24GiB
1T. Obtaining a li...      1    499ms    1.1%   499ms   52.5MiB    3.1%  52.5MiB
────────────────────────────────────────────────────────────────────────────────
Sequential contraction result: fill(0.7071067811862335 + 0.0im)


---

## Step 3: Compare Results Using Another Contraction Plan
Here, we utilize FlowCutter to generate an alternative contraction plan and compare results.

In [9]:
# --- Step 3: Compare with Another Plan ---
using QXTools
using QXTools.Circuits

 tnc = convert_to_tnc(cct)  # Convert the GHZ circuit into a tensor network circuit

# Find a good contraction plan using FlowCutter
plan = flow_cutter_contraction_plan(tnc; time=10)

num_qubits = cct.num_qubits

# Output states
outputs = ["0" ^ num_qubits, "1" ^ num_qubits, "1" ^ (num_qubits - 2) * "01"]
eixida = outputs[1]  # Select the first output as the target

# Evaluate the probability amplitude for different outputs

@show QXTools.single_amplitude(tnc, plan, eixida)
eixida = outputs[2]  # Select the second output
@show QXTools.single_amplitude(tnc, plan, eixida)
eixida = outputs[3]  # Select the third output
@show QXTools.single_amplitude(tnc, plan, eixida)

# Perform tensor network contraction using the plan
s = contract_tn!(tnc.tn, plan)
println("Contraction result using FlowCutter plan: ", s)

# Compare results
println("Do the results match? ", s1 ≈ s)

QXTools.single_amplitude(tnc, plan, eixida) = 0.7071067811862398 + 0.0im
QXTools.single_amplitude(tnc, plan, eixida) = 0.7071067811862679 + 0.0im
QXTools.single_amplitude(tnc, plan, eixida) = 0.0 + 0.0im
Contraction result using FlowCutter plan: fill(0.7071067811862398 + 0.0im)
Do the results match? true


---

## Step 4: Parallel Contraction Using ComPar
Finally, we use ComPar algorithms for parallel contraction.

In [10]:
# --- Step 4: Parallel Contraction using ComPar ---
# Define input and output states
num_qubits = cct.num_qubits
entrada = "0" ^ num_qubits

# Output states
outputs = ["0" ^ num_qubits, "1" ^ num_qubits, "1" ^ (num_qubits - 2) * "01"]
eixida = outputs[1]  # Select the first output as the target

n_com = 8  # Number of communities for the contraction
println("Using $n_com communities for contraction.")



Using 8 communities for contraction.


In [11]:
# Perform contraction using ComParCPU
s2 = ComParCPU(cct, entrada, eixida, n_com; timings=true, decompose=true)
println("Contraction result using ComParCPU: ", s)

────────────────────────────────────────────────────────────────────────────────
                                       Time                    Allocations      
                              ───────────────────────   ────────────────────────
      Tot / % measured:            83.5s / 100.0%            525MiB / 100.0%    

Section               ncalls     time    %tot     avg     alloc    %tot      avg
────────────────────────────────────────────────────────────────────────────────
1T.Obtaining Commu...      1    77.5s   92.8%   77.5s   63.5MiB   12.1%  63.5MiB
2T.Parallel contra...      1    6.02s    7.2%   6.02s    461MiB   87.8%   461MiB
3T.Final Contraction       1   15.2ms    0.0%  15.2ms    376KiB    0.1%   376KiB
────────────────────────────────────────────────────────────────────────────────
Contraction result using ComParCPU: fill(0.7071067811862398 + 0.0im)


In [12]:
# Perform contraction using ComParCPU_para
s_para = ComParCPU_para(cct, entrada, eixida, n_com; timings=true, decompose=true)
println("Contraction result using ComParCPU_para: ", s_para)

────────────────────────────────────────────────────────────────────────────────
                                       Time                    Allocations      
                              ───────────────────────   ────────────────────────
      Tot / % measured:            82.1s / 100.0%            497MiB / 100.0%    

Section               ncalls     time    %tot     avg     alloc    %tot      avg
────────────────────────────────────────────────────────────────────────────────
1T.Obtaining Commu...      1    77.9s   94.9%   77.9s   46.3MiB    9.3%  46.3MiB
2T.Parallel contra...      1    4.08s    5.0%   4.08s    449MiB   90.5%   449MiB
3T.Final contracti...      1    108ms    0.1%   108ms   1.01MiB    0.2%  1.01MiB
────────────────────────────────────────────────────────────────────────────────
Contraction result using ComParCPU_para: fill(0.7071067811862335 + 0.0im)


In [13]:
# Compare results
println("Do all the results match? ", s1 ≈ s≈ s2 ≈ s_para)

Do all the results match? true


## Step 5: Improving contractions using a diferent algorithm for obtainig communities

In these functions we use a Fast Greedy algorithm for getting communities faster. 

In [14]:
sol_fast = ComParCPU_GHZ(cct,entrada,eixida;timings=true,decompose=true)

────────────────────────────────────────────────────────────────────────────────
                                       Time                    Allocations      
                              ───────────────────────   ────────────────────────
      Tot / % measured:            1.89s / 100.0%            472MiB / 100.0%    

Section               ncalls     time    %tot     avg     alloc    %tot      avg
────────────────────────────────────────────────────────────────────────────────
2T.Parallel contra...      1    1.52s   80.4%   1.52s    421MiB   89.2%   421MiB
1T.Obtaining Commu...      1    344ms   18.2%   344ms   46.4MiB    9.8%  46.4MiB
3T.Final contraction       1   25.6ms    1.4%  25.6ms   4.53MiB    1.0%  4.53MiB
────────────────────────────────────────────────────────────────────────────────


0-dimensional Array{ComplexF64, 0}:
0.7071067811862488 + 0.0im

In [15]:
sol_para = ComParCPU_para_GHZ(cct,entrada,eixida;timings=true,decompose=true)

────────────────────────────────────────────────────────────────────────────────
                                       Time                    Allocations      
                              ───────────────────────   ────────────────────────
      Tot / % measured:            2.43s / 100.0%            472MiB / 100.0%    

Section               ncalls     time    %tot     avg     alloc    %tot      avg
────────────────────────────────────────────────────────────────────────────────
2T.Parallel contra...      1    2.02s   83.1%   2.02s    421MiB   89.2%   421MiB
1T.Obtaining Commu...      1    371ms   15.3%   371ms   46.4MiB    9.8%  46.4MiB
3T.Final contracti...      1   39.8ms    1.6%  39.8ms   4.54MiB    1.0%  4.54MiB
────────────────────────────────────────────────────────────────────────────────


0-dimensional Array{ComplexF64, 0}:
0.7071067811862488 + 0.0im

In [16]:
# Compare results
println("Do all the results match? ", s1 ≈ s≈ s2 ≈ s_para≈ sol_fast≈ sol_para)

Do all the results match? true


---

### Summary
This notebook demonstrated:
1. The creation of a GHZ tensor network.
2. Sequential contraction using the Girvan–Newman algorithm.
3. Alternative contraction using FlowCutter.
4. Parallel contraction with ComParCPU and ComParCPU_para.
5. Using a new algorithm for generating communities can improve contraction time.

Thank you for exploring tensor network contraction with us!